<a href="https://colab.research.google.com/github/LidiiaMelnyk95/FSU_Jena_scripts/blob/main/seq2seq_gec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/replacement_spelling_deduplicated-2.csv', sep =';')

***Data Augmentation***

In [ ]:
import pandas as pd

def apply_spelling_replacement(comment, replacements):
    try:
        for original, replacement in replacements.items():
            comment = comment.replace(original, replacement)
        return comment
    except AttributeError:
        return None

replacement_trans = {'Trans', 'trans'}
replacements_dass = {'dass': 'daß'}
replacements_umlaute = {'ä': 'ae', 'ö': 'oe', 'ü': 'ue', 'Ä': 'Ae', 'Ö': 'Oe', 'Ü': 'Ue'}
replacements_comma = {',': ''}

# Apply spelling replacements
for replacements in [replacements_dass, replacements_umlaute, replacements_comma]:
    df['SPELLING'] = [apply_spelling_replacement(comment, replacements) for comment in df['Comment'][::5]]
    df = pd.concat([df, pd.DataFrame({'SPELLING': df['SPELLING'], 'Comment': df['Comment'][::5]})], ignore_index=True)

df.shape
df.to_csv('data_augmented_df.csv')


(1876, 3)

added 972 examples

In [ ]:
df['encoder_input'] = df["SPELLING"]
df["decoder_output"] = df["Comment"]

In [ ]:
df.head()

,Unnamed: 0,SPELLING,Comment,encoder_input,decoder_output
0,1543.0,:) Bei Partner:innenakrobatikfestivals stapeln...,:) Bei Partner:innenakrobatikfestivals stapel...,:) Bei Partner:innenakrobatikfestivals stapeln...,:) Bei Partner:innenakrobatikfestivals stapel...
1,2558.0,:D okay ... deswegen geht einem die Regenboge...,:D okay ... deswegen geht einem die Regenboge...,:D okay ... deswegen geht einem die Regenboge...,:D okay ... deswegen geht einem die Regenboge...
2,679.0,"""Das Naturgesetz"" lmaoIch hoffe du trägst kei...","""Das Naturgesetz"" lmaoIch hoffe du trägst kei...","""Das Naturgesetz"" lmaoIch hoffe du trägst kei...","""Das Naturgesetz"" lmaoIch hoffe du trägst kei..."
3,2290.0,"""ja schön. Diese Person kann sich ja auch dem...","""ja schön. Diese Person kann sich ja auch dem...","""ja schön. Diese Person kann sich ja auch dem...","""ja schön. Diese Person kann sich ja auch dem..."
4,822.0,"""Meine Seele"", ""mein Körper"" ""ich"", ""ich"", .....","""Meine Seele"", ""mein Körper"" ""ich"", ""ich"", .....","""Meine Seele"", ""mein Körper"" ""ich"", ""ich"", .....","""Meine Seele"", ""mein Körper"" ""ich"", ""ich"", ....."


In [ ]:
df['decoder_input'] = df['decoder_output'].apply(lambda x: '<start>' + ' ' + x + ' ' +  '<end>')
df_big = df[['encoder_input', 'decoder_input', 'decoder_output']]

In [ ]:
df_big

,encoder_input,decoder_input,decoder_output
0,:) Bei Partner:innenakrobatikfestivals stapeln...,<start> :) Bei Partner:innenakrobatikfestival...,:) Bei Partner:innenakrobatikfestivals stapel...
1,:D okay ... deswegen geht einem die Regenboge...,<start> :D okay ... deswegen geht einem die R...,:D okay ... deswegen geht einem die Regenboge...
2,"""Das Naturgesetz"" lmaoIch hoffe du trägst kei...","<start> ""Das Naturgesetz"" lmaoIch hoffe du tr...","""Das Naturgesetz"" lmaoIch hoffe du trägst kei..."
3,"""ja schön. Diese Person kann sich ja auch dem...","<start> ""ja schön. Diese Person kann sich ja ...","""ja schön. Diese Person kann sich ja auch dem..."
4,"""Meine Seele"", ""mein Körper"" ""ich"", ""ich"", .....","<start> ""Meine Seele"", ""mein Körper"" ""ich"", ""...","""Meine Seele"", ""mein Körper"" ""ich"", ""ich"", ....."
...,...,...,...
308,Ich ging die ganze Zeit davon aus dass dieses ...,"<start> Ich ging die ganze Zeit davon aus, das...","Ich ging die ganze Zeit davon aus, dass dieses..."
309,Meinst du die können dann theoretisch Olympia ...,<start> Meinst du die können dann theoretisch ...,Meinst du die können dann theoretisch Olympia ...
310,Transgendersoldaten an die Front das wird der ...,<start> Transgendersoldaten an die Front das w...,Transgendersoldaten an die Front das wird der ...
311,„Ich bin ja nicht transfeindlich aber hat im F...,"<start> „Ich bin ja nicht transfeindlich, aber...","„Ich bin ja nicht transfeindlich, aber hat im ..."


In [ ]:
df_big = df_big.sample(frac = 1).reset_index()

***Training Preparation***

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
train, test = train_test_split(df_big, test_size=0.10)

In [ ]:
import pickle
import numpy as np
import string
from keras.preprocessing.text import Tokenizer
import gensim.downloader as api

def preprocess_text_with_punctuation(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    return text.translate(translator)

def fit_tokenizer(texts, filters=''):
    tokenizer = Tokenizer(filters=filters, lower=False)
    tokenizer.fit_on_texts(texts)
    return tokenizer

def create_embedding_matrix(tokenizer, embeddings_index):
    embedding_dim = len(embeddings_index.get(embeddings_index.index_to_key[0]))
    words_and_punctuation = list(tokenizer.word_index.keys()) + list(string.punctuation)
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

    for word, i in tokenizer.word_index.items():
        if word in words_and_punctuation:
            # Handle hyphens in word embeddings
            word = word.replace('-', '_')
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

    return embedding_matrix

# Load the German GloVe model
model = api.load("glove-twitter-25")

# Preprocess the text to include punctuation as separate tokens
train['encoder_input'] = train['encoder_input'].apply(preprocess_text_with_punctuation)
train['decoder_input'] = train['decoder_input'].apply(preprocess_text_with_punctuation)

# Fit the tokenizers on the preprocessed data
token_perturbation = fit_tokenizer(list(train['encoder_input'].values) + list(string.punctuation), filters='')
token_correct = fit_tokenizer(list(train['decoder_input'].values) + list(string.punctuation), filters='')

# Save tokenizers using pickle
with open("token_perturbation.pkl", "wb") as f:
    pickle.dump(token_perturbation, f)

with open("token_correct.pkl", "wb") as f:
    pickle.dump(token_correct, f)

# Create embedding matrices
embedding_matrix_perturbation = create_embedding_matrix(token_perturbation, model)
embedding_matrix_correct = create_embedding_matrix(token_correct, model)

# Print the shapes of the embedding matrices
print(f"Embedding Matrix Shape for perturbation: {embedding_matrix_perturbation.shape}")
print(f"Embedding Matrix Shape for correct: {embedding_matrix_correct.shape}")


9232
8280


In [ ]:
pickle.dump(embedding_matrix, open('embedding_matrix.pkl','wb'))


In [ ]:
pickle.dump(embedding_matrix_per, open('embedding_matrix_per.pkl','wb'))


***Model Architecture***

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, enc_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0

    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder", input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self, input_sentances, training=True):
        input_embedd                        = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c

In [ ]:
encoder_vanilla = pickle.dump(Encoder, open('encodv.pkl','wb'))


In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = 25
        self.dec_units = dec_units
        self.input_length = input_length

    def build(self, input_shape):
        # we are using embedding_matrix weights and not training the embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix],input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")

    def call(self, target_sentances, state_h, state_c):
        target_embedd           = self.embedding(target_sentances)
        lstm_output, _,_        = self.lstm(target_embedd, initial_state=[state_h, state_c])
        return lstm_output

In [ ]:
class Dataset:
    def __init__(self, data, token_perturbation, token_correct, max_len):
        self.encoder_inps = data['encoder_input'].values
        self.decoder_inps = data['decoder_input'].values
        self.decoder_outs = data['decoder_output'].values
        self.token_correct = token_correct
        self.token_perturbation = token_perturbation
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.token_perturbation.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.token_correct.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.token_correct.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int64', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int64', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int64', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

class Dataloader(tf.keras.utils.Sequence):

    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]

        # we are creating data like ([perturbation, correct_inp], correct_out) these are already converted into seq

        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [ ]:
class vanilla_model(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__()
        self.encoder = Encoder(vocab_size=v_perturbation, embedding_dim=25, input_length=encoder_inputs_length, enc_units=512)
        self.decoder = Decoder(vocab_size=v_correct, embedding_dim=25, input_length=decoder_inputs_length, dec_units=512)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        self.dropout = tf.keras.layers.Dropout(0.01)
        self.normalization =  tf.keras.layers.LayerNormalization(epsilon=0.005)


    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(tf.cast(output[:8869], tf.int64), encoder_h, encoder_c)
        decoder_output = self.normalization(decoder_output)
        decoder_output = self.dropout(decoder_output)
        output                               = self.dense(decoder_output)
        return output

***Model evaluation metrics***

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
from tensorflow.keras.callbacks import LambdaCallback
metrics_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: print(compute_metrics(val_data, model.predict(val_data))))


***Model Training***

In [ ]:
train_dataset = Dataset(train, token_perturbation, token_correct,100 )
test_dataset = Dataset(test, token_perturbation, token_correct, 100)

train_dataloader = Dataloader(train_dataset, batch_size=100)
test_dataloader = Dataloader(test_dataset, batch_size=100)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(100, 100) (100, 100) (100, 100)


In [ ]:
vanilla = vanilla_model(encoder_inputs_length=512, decoder_inputs_length=512, output_vocab_size=v_perturbation)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0007)
vanilla.compile(optimizer= optimizer, loss= loss_function, metrics=['accuracy'])

In [ ]:
import datetime

logfile = "/content//Project_2/logs/vanilla/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#logfile = "C:\\Users\\rpris\\Google Drive\\Project_2\\logs\\vanilla\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tfboard = tf.keras.callbacks.TensorBoard(log_dir=logfile, histogram_freq=1, write_graph=True)

chkfile = "/content/Project_2/wts/vanilla/weights-{epoch:02d}-{val_loss:.4f}.hdf5"
#chkfile2 = "C:\\Users\\rpris\\Google Drive\\Project_2\\wts\\vanilla\\weights-{epoch:02d}-{val_loss:.4f}.hdf5"
chkpt = tf.keras.callbacks.ModelCheckpoint(chkfile, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=0, mode='min')

stp = tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss')

In [ ]:
train_steps=train.shape[0]//16
vanilla.fit(train_dataloader,  epochs =50)


Epoch 1/50
16/16 [==============================] - 17s 407ms/step - loss: 2.5402 - accuracy: 0.2440
Epoch 2/50
16/16 [==============================] - 3s 164ms/step - loss: 2.1466 - accuracy: 0.2624
Epoch 3/50
16/16 [==============================] - 3s 167ms/step - loss: 2.0594 - accuracy: 0.2662
Epoch 4/50
16/16 [==============================] - 3s 170ms/step - loss: 1.9942 - accuracy: 0.2714
Epoch 5/50
16/16 [==============================] - 3s 168ms/step - loss: 2.0221 - accuracy: 0.2711
Epoch 6/50
16/16 [==============================] - 3s 167ms/step - loss: 1.9871 - accuracy: 0.2717
Epoch 7/50
16/16 [==============================] - 3s 166ms/step - loss: 1.9837 - accuracy: 0.2705
Epoch 8/50
16/16 [==============================] - 3s 170ms/step - loss: 2.1423 - accuracy: 0.2622
Epoch 9/50
16/16 [==============================] - 3s 165ms/step - loss: 2.0718 - accuracy: 0.2647
Epoch 10/50
16/16 [==============================] - 3s 163ms/step - loss: 1.9824 - accuracy: 0.268

In [ ]:
from tensorflow.keras.callbacks import Callback
from rouge import Rouge

# Define a custom callback to compute Rouge score during training
class RougeScoreCallback(Callback):
    def __init__(self, encoder_input_val, decoder_input_val, target_val):
        self.encoder_input_val = encoder_input_val
        self.decoder_input_val = decoder_input_val
        self.target_val = target_val
        self.rouge = Rouge()

    def on_epoch_end(self, epoch, logs=None):
        translations = []
        targets = []
        for i in range(len(self.encoder_input_val)):
            enc_input = self.encoder_input_val[i]
            dec_input = self.decoder_input_val[i]
            target = self.target_val[i]
            translation = inference(enc_input, dec_input)
            translations.append(translation)
            targets.append(target)
        scores = self.rouge.get_scores(translations, targets, avg=True)
        logs = logs or {}
        logs['rouge-1'] = scores['rouge-1']['f']
        logs['rouge-2'] = scores['rouge-2']['f']
        logs['rouge-l'] = scores['rouge-l']['f']
        print(f"Rouge-1: {logs['rouge-1']:.4f} - Rouge-2: {logs['rouge-2']:.4f} - Rouge-L: {logs['rouge-l']:.4f}")
        return logs

# Instantiate the custom callback
rouge_score_callback = RougeScoreCallback(encoder_input_val, decoder_input_val, target_val)

# Train the model with the custom callback
vanilla.fit(train_dataloader, steps_per_epoch=train_steps, epochs=200, validation_data=(test_dataloader), callbacks=[rouge_score_callback])


In [ ]:
vanilla.build([512,32,256])

In [ ]:
output_test = vanilla.predict(test_dataloader)

In [ ]:
some_list =list(token_correct.word_index.keys())

***Model Inference***

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def inference(enc_inp, dec_inp):
    translation = ""

    # Convert encoder input to sequences and pad
    e_input = pad_sequences(token_perturbation.texts_to_sequences([enc_inp]), maxlen=16, padding='post')

    # Get encoder output and states
    e_output, e_hidden, e_cell = vanilla.layers[0](e_input)

    # Convert decoder input to sequences and pad
    d_input = pad_sequences(token_correct.texts_to_sequences([dec_inp]), maxlen=16, padding='post')

    # Generate prediction
    prediction = vanilla.layers[2](vanilla.layers[1](d_input, e_hidden, e_cell))

    # Retrieve words from indices in the prediction
    for word_index in tf.argmax(prediction, axis=2).numpy()[0]:
        try:
            word = token_correct.index_word[word_index]
            translation += word + " "
        except KeyError:
            pass

    return translation


In [ ]:
train.head()

,index,encoder_input,decoder_input,decoder_output
541,118,Ist es wenn man sich die Definition von transs...,< start > Ist es wenn man sich die Definitio...,Ist es wenn man sich die Definition von Transs...
432,238,"Ich ging die ganze Zeit davon aus , dass dies...",< start > Ich ging die ganze Zeit davon aus ...,"Ich ging die ganze Zeit davon aus, dass dieses..."
792,661,"Mulm , damit hätte ich nicht gerechnet . Vie...","< start > Mulm , damit hätte ich nicht gere...","Mulm, damit hätte ich nicht gerechnet. Vielen ..."
637,327,der rapide anstieg ist allerdings ein ziemlic...,< start > Der rapide Anstieg ist allerdings ...,Der rapide Anstieg ist allerdings ein ziemlich...
1687,162,War ja klar dass diese Scheiße gut findet . ....,< start > War ja klar dass diese Scheiße gut...,War ja klar dass diese Scheiße gut findet..gut...


In [ ]:
sample_train = train.sample(100)
sample_train['pred'] = sample_train.apply(lambda row: inference(str(row['encoder_input']), str(row['decoder_output'])), axis=1)

sample_train['correct_output'] = sample_train['decoder_output']

sample_train.drop(['decoder_input', 'decoder_output'], axis=1, inplace=True)

sample_train.head(10)


Trotzdem die Psyche von denen lässt aber sich sich sich sich die auch bewusst die nicht es ist eben zu ist ist ist 
Der Der kontra Einfach finde jeder Weil für der kann ich kann dann es es es ich welche die die das Fälle stimmt stimmt mal wenn wenn wenn tatsächlich tatsächlich tatsächlich ich ich ich Aufwand lieber bin bin bin bin bin mir mir mir mir mir mir mir mir mir geben mir und und und und und und und und 
Godfather die dir ganz sprechen ganz Viele beachten ist ist in dieser Zeit dieser dieser dieser vor seinen ihre 
Der die die die die die 
kontra der dran Perspektive mit mit mit nachdem ich ich ich nicht nicht nicht nicht durch ziemlich von Hormonbehandlungen lange lange lange lange lange wie weshalb was leider gesagt was zum zum und zum der und sich sich Am sich sich sich der und Wenn und ganz Angst Angst und ich ich ich ich sehr neuen es dem dem dem kann Namen mir mir mir mir neuen heute dem dem nur nur gut Diskussion der der der der von von Diskussion Ende Ende sein dass woh

,index,encoder_input,pred,correct_output
991,156,Trotzdem sollte man davor die Psyche von denen...,Trotzdem die Psyche von denen lässt aber sich ...,Trotzdem sollte man davor die Psyche von denen...
126,3,""" ja schön . Diese Person kann sich ja auch...",Der Der kontra Einfach finde jeder Weil für de...,"""ja schön. Diese Person kann sich ja auch dem..."
847,740,", sie sind ein Mensch der Kultur und sprech...",Godfather die dir ganz sprechen ganz Viele bea...,Sie sind ein Mensch der Kultur und sprechen in...
1472,31,""" Ihr klaut den Patienten wichtige Buchstaben...",Der die die die die die,"""Ihr klaut den Patienten wichtige Buchstaben.""..."
1874,525,"Ich ging die ganze Zeit davon aus , dass dies...",kontra der dran Perspektive mit mit mit nachde...,"Ich ging die ganze Zeit davon aus, dass dieses..."
1636,13,"Gerne , bitte : )",Der ich ich,"Gerne, bitte :)"
704,346,Die neue Melanie - Endlich Leben interessant...,Größe Seite total Weil bitte In Thema In Thema...,Die neue Melanie - Endlich Leben interessanter...
1312,158,""" Statistisch gesehen sollte es in jeder Klas...",Der Ich und und chirurgisch und es Wenn der de...,"""Statistisch gesehen sollte es in jeder Klasse..."
1444,193,35 : 40 Wenn man keinen Diskurs mit der Gegens...,Der sie das es ist von die der sich sich sich ...,35:40 Wenn man keinen Diskurs mit der Gegensei...
599,40,"94 daß ist mir durchaus bewusst , aber die Au...",94 dass die viel ich ich ich kann ich ich kann...,"94 dass ist mir durchaus bewusst, aber die Aus..."


***Additional evaluation***

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.corpus import wordnet as wn

def precision(candidate, reference, n):
    """
    Calculate the precision of n-grams in a text.

    Args:
    - candidate (str): The generated text
    - reference (str): The reference text
    - n (int): The n-gram order

    Returns:
    - float: The precision score
    """
    candidate_ngrams = ngrams(word_tokenize(candidate), n)
    reference_ngrams = ngrams(word_tokenize(reference), n)
    candidate_ngrams_set = set(candidate_ngrams)
    reference_ngrams_set = set(reference_ngrams)
    common_ngrams = candidate_ngrams_set.intersection(reference_ngrams_set)
    precision = len(common_ngrams) / len(candidate_ngrams_set)
    return precision

In [ ]:
sample_train['precision'] = sample_train.apply(lambda row: brevity_penalty(row['pred'], row['correct_output'], 2) if len(row['correct_output']) > 0 else None, axis=1)

mean_precision = sample_train['precision'].mean(skipna=True)
print(mean_precision)


In [ ]:
def brevity_penalty(candidate, reference):
    """
    Calculate the brevity penalty for the precision score.

    Args:
    - candidate (str): The generated text
    - reference (str): The reference text

    Returns:
    - float: The brevity penalty
    """
    candidate_length = len(word_tokenize(candidate))
    reference_length = len(word_tokenize(reference))
    if candidate_length > reference_length:
        brevity_penalty = 1
    else:
        brevity_penalty = np.exp(1 - reference_length / candidate_length)
    return brevity_penalty

In [ ]:
sample_train['brevity_penalty'] = sample_train.apply(lambda row: brevity_penalty(row['pred'], row['correct_output'], 2) if len(row['correct_output']) > 0 else None, axis=1)

mean_brevity_penalty = sample_train['brevity_penalty'].mean(skipna=True)
print(mean_brevity_penalty)


In [ ]:
def gleu(candidate, reference, max_order=4):
    """
    Calculate the GLEU score for a generated text compared to a reference text.

    Args:
    - candidate (str): The generated text
    - reference (str): The reference text
    - max_order (int): The maximum n-gram order to consider (default: 4)

    Returns:
    - float: The GLEU score
    """
    precision_scores = []
    for n in range(1, max_order + 1):
        precision_scores.append(precision(candidate, reference, n))
    brevity_penalty_score = brevity_penalty(candidate, reference)
    gleu_score = brevity_penalty_score * np.exp(np.mean(np.log(precision_scores)))
    return gleu_score

In [ ]:
sample_train['gleu'] = sample_train.apply(lambda row: gleu(row['pred'], row['correct_output'], 2) if len(row['correct_output']) > 0 else None, axis=1)

mean_gleu = sample_train['gleu'].mean(skipna=True)
print(mean_gleu)


<ipython-input-58-4694358a9c7a>:17: RuntimeWarning: divide by zero encountered in log
  gleu_score = brevity_penalty_score * np.exp(np.mean(np.log(precision_scores)))


0.03317760027044629
